<a href="https://colab.research.google.com/github/dioday45/CS433_Project2/blob/jeremy/notebooks/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 10.4 MB/s 
     |████████████████████████████████| 182 kB 34.2 MB/s 
     |████████████████████████████████| 7.6 MB 59.5 MB/s 


In [7]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import pandas as pd
from scipy.special import softmax
import torch
tqdm.pandas()

In [8]:
device = 'cuda'


In [ ]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)
model.to(device)
text = "Covid cases are increasing fast!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt').to(device)
output = model(**encoded_input)
scores = output[0][0]
scores = softmax(scores.cpu().detach().numpy())


Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) negative 0.7236
2) neutral 0.2287
3) positive 0.0477


In [ ]:
t_pos = pd.read_table('train_pos.txt', header=None, names=['tweet'], dtype=str,on_bad_lines='skip')
t_pos['label'] = 1
t_neg = pd.read_table('train_neg.txt', header=None, names=['tweet'], dtype=str,on_bad_lines='skip')
t_neg['label'] = -1
df = pd.concat((t_pos,t_neg), ignore_index=True)

In [ ]:
X = df['tweet']
y = df['label']
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.2, random_state =26105111)

In [ ]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt').to(device)
  result = model(tokens)
  return int(torch.argmax(result.logits))



X_val['sentiment'] = X_val.progress_apply(lambda x: sentiment_score(x[:512]))

100%|██████████| 39394/39394 [08:15<00:00, 79.54it/s]


In [ ]:
X_val['sentiment']

151196    0
88742     2
37122     0
9124      2
48358     2
         ..
166308    0
172111    1
55543     2
101263    1
143314    1
Name: tweet, Length: 39394, dtype: int64

In [2]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.999840259552002}]

In [19]:
t_pos = pd.read_table('train_pos.txt', header=None, names=['tweet'], dtype=str,on_bad_lines='skip')
t_pos['label'] = 1
t_neg = pd.read_table('train_neg.txt', header=None, names=['tweet'], dtype=str,on_bad_lines='skip')
t_neg['label'] = -1
df = pd.concat((t_pos,t_neg), ignore_index=True)

In [20]:
X = df['tweet']
y = df['label']
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.05, random_state =26105111)

In [22]:
X_val['sentiment'] = X_val.progress_apply(lambda x: classifier(x)).to(device)

  7%|▋         | 738/9849 [00:45<09:19, 16.30it/s]


RuntimeError: ignored

In [24]:
X_val

KeyError: ignored